## 1. Cargamos los datos a Preprocesar y limpiar (desde el BlobStorage)

In [0]:
#Librerias necesarias para usar el Azure DataLake y DataBrinks
import pandas as pd
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient
import io

# 2. Obtener conection Azure DataLake,interfaz de Azure(Claves de acceso: Key1) (Debes comentar esta variable Si NO deja hacer COMMIT DEL 
# CODIGO EN GIT,GITHUB)
#connection_string = 'DefaultEndpointsProtocol=https;AccountName=datalakemlopsd4m;AccountKey=iWT8t74/#XlqcqoR03keDVtFZPzr0PB9zDffMPaLWMUBIAjUww8uYAVkc9xRkcBtvTmUHKBvd1sB3+ASt6mGgcQ==;EndpointSuffix=core.windows.net'

# 3. Conectar al Blob Storage de Azure
blob_service_client = BlobServiceClient.from_connection_string(connection_string)

# 4. Identificamos el nombre del contenedor(container_name) y nombre del archivo(definir blob_name) en el Blob Storage
container_name = "raw/proyectocongestion_raw/fuentedatos_consolidado/"
blob_name = "datos_raw_consolidado.csv"

# 5. Obtener el blob_client
blob_client = blob_service_client.get_blob_client(container=container_name, blob=blob_name)

# 6. Leer el contenido del blob como texto
#blob_data = blob_client.download_blob().content_as_text()
blob_data = blob_client.download_blob().readall().decode('utf-8')

# Especificar tipos de datos al leer el archivo CSV
column_types = {
    'instant_date_t': 'datetime64[ns]',
    'Event Date' : 'datetime64[ns]'
}

# Leer el archivo CSV en un DataFrame de Pandas
datos = pd.read_csv(io.StringIO(blob_data), parse_dates=['instant_date_t', 'Event Date'])

# 7. Leer el archivo CSV en un DataFrame de Pandas desde el texto
#datos = pd.read_csv(io.StringIO(blob_data))
datos.head()

In [0]:
datos.dtypes

In [0]:
datos[["instant_date_t", "Event Date"]].head(15)

In [0]:
datos["eq_id"].unique()

In [0]:
#Librerias que se usaran
import pandas as pd  # Libreria para administrar tablas, y realizar trabajos con distintas formas de tablas o dataframes
import numpy as np   # Libreria para poder hacer operaciones matematicas y matriciales
import matplotlib.pyplot as plt # Libreria para realizar graficos 

#Configuramos pandas para que podamos vizualizar todas las columnas y filas la estadistica descriptiva de todas las variables
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)
#Configuramos pandas para que lanze valores con una precision de hasta 6 decimales
pd.set_option('display.float_format', '{:.6f}'.format)

In [0]:
datos.head()

In [0]:
import plotly.express as px

# Obtiene las dimensiones de tu conjunto de datos
num_filas, num_columnas = datos.shape

# Crea un DataFrame con los datos de las dimensiones
df_dimension = pd.DataFrame({'Dimensión': ['Filas', 'Columnas'], 'Valor': [num_filas, num_columnas]})

# Crea un gráfico de pastel interactivo con etiquetas
fig = px.pie(df_dimension, names='Dimensión', values='Valor', title='Dimensiones del Conjunto de Datos')

# Agrega etiquetas y porcentajes
fig.update_traces(textinfo='label+percent')

# Muestra el gráfico interactivo
fig.show()

In [0]:
datos.dtypes

#### Obtener equipos de camión únicos y sus nombres único

In [0]:
# Obtener equipos de camión únicos y sus nombres únicos
equipos_camiones = datos.loc[datos["eq_id"].notnull(), ["eq_id", "nombre_x"]] \
    .drop_duplicates(subset="eq_id")
print(equipos_camiones)

#### Obtener por equipos, la velocidad en promedio

In [0]:
datos.groupby('eq_id')['speed_t'].mean()

#### Obtener por equipos, y por horas, el promedio de la velocidad

In [0]:
# Extraer la hora y crear una nueva columna para almacenarla
datos['hora'] = datos['instant_date_t'].dt.hour

# Agrupar por id_equipo y hora, calcular el promedio de velocidad
datos.groupby(['eq_id', 'hora'])['speed_t'].mean()

In [0]:
import matplotlib.pyplot as plt

# Agrupar por id_equipo y hora, calcular el promedio de velocidad
promedios_velocidad = datos.groupby(['eq_id', 'hora'])['speed_t'].mean().reset_index()

# Crear el gráfico de líneas
plt.figure(figsize=(10, 6))
for id_equipo, group in promedios_velocidad.groupby('eq_id'):
    plt.plot(group['hora'], group['speed_t'], label=f'Equipo {id_equipo}')

# Añadir etiquetas y título
plt.xlabel('Hora')
plt.ylabel('Promedio de Velocidad')
plt.title('Promedio de Velocidad por Hora y Equipo')
plt.legend()
plt.grid(True)

# Mostrar el gráfico
plt.show()
